In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import geemap
import geetools
from datetime import datetime
import pytz

In [2]:
ee.Initialize(project='ee-jonstar')

In [3]:
#ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [4]:
edt = pytz.timezone('US/Eastern')
#mdt = pytz.timezone('US/Mountain')
az = pytz.timezone('US/Arizona')

"""
Convert datetimes from your current timezone to a desired timezone.

dt (datetime): python datetime object
to_timezone (pytz timezone): timezone to convert to
local_timezone (pytz timezone): the timezone you are currently in. Python uses your computer's internal clock, so whatever your computer is on.
"""
def toTimezone(dt, to_timezone, local_timezone=edt):
    return local_timezone.normalize(local_timezone.localize(dt)).astimezone(to_timezone)

## CONUS Geometry

In [5]:
conus = ee.ImageCollection("GRIDMET/DROUGHT").filterDate('2000-01-01', '2001-01-01')
geometry = conus.geometry();

In [41]:
"""
Points of interest for each urban area.
KEY:
First and second points = Landsat
Third point = Sentinel
If the second point is zero, then there is only one Landsat image needed to cover that city and its surrounding area
The third point is defined near the city center. A circular region of 200 km is created around the third point
to filter Sentinel-1 images, ensuring all of the necessary images will be present.
"""
city_points = {
    'DMV':[ee.Geometry.Point(-76.6122, 39.2904), 0, ee.Geometry.Point(-76.863, 39.063)],
    'NYC':[ee.Geometry.Point(-73.274, 40.688), 0, ee.Geometry.Point(-73.979, 40.751)],
    'Atlanta':[ee.Geometry.Point(-84.039, 34.155), ee.Geometry.Point(-84.485, 33.641), ee.Geometry.Point(-84.38, 33.748)],
    'Miami':[ee.Geometry.Point(-80.96, 26.358), 0, ee.Geometry.Point(-80.267, 25.818)],
    'Chicago':[ee.Geometry.Point(-88.037, 41.826), 0, ee.Geometry.Point(-87.74, 41.78)],
    'Phoenix':[ee.Geometry.Point(-112.02, 33.46), 0, ee.Geometry.Point(-112.064, 33.487)],
    'Denver':[ee.Geometry.Point(-104.6, 39.6696), 0, ee.Geometry.Point(-104.985, 39.729)],
    'Seattle':[ee.Geometry.Point(-121.952, 47.546), 0, ee.Geometry.Point(-122.322, 47.59)],
    'San_Francisco':[ee.Geometry.Point(-122.179, 37.745), 0, ee.Geometry.Point(-122.365, 37.745)],
    'Los_Angeles':[ee.Geometry.Point(-118.365, 34.279), ee.Geometry.Point(-118.744, 33.383), ee.Geometry.Point(-122.365, 37.745)],
    'Toronto':[ee.Geometry.Point(-80.273, 43.289), ee.Geometry.Point(-79.609, 44.187), ee.Geometry.Point(-79.46, 43.616)],
    'Mexico_City':[ee.Geometry.Point(-99.101, 19.012), ee.Geometry.Point(-99.187, 20.084), ee.Geometry.Point(-99.129, 19.422)],
    'Las_Vegas':[ee.Geometry.Point(-115.162, 36.154), 0, ee.Geometry.Point(-115.161, 36.154)],
    'Salt_Lake_City':[ee.Geometry.Point(-111.896, 40.752), ee.Geometry.Point(-111.57, 41.37), ee.Geometry.Point(-111.896, 40.752)],
    'Dallas':[ee.Geometry.Point(-96.858, 32.789), 0, ee.Geometry.Point(-96.968, 32.794)],
    'Houston':[ee.Geometry.Point(-94.816, 30.034), ee.Geometry.Point(-95.356, 29.399), ee.Geometry.Point(-95.376, 29.758)],
    'New_Orleans':[ee.Geometry.Point(-90.102, 29.947), 0, ee.Geometry.Point(-90.102, 29.947)],
    'St_Louis':[ee.Geometry.Point(-90.505, 38.644), 0, ee.Geometry.Point(-90.205, 38.624)],
    'Minneapolis':[ee.Geometry.Point(-93.248, 44.969), 0, ee.Geometry.Point(-93.248, 44.969)],
    'Jacksonville':[ee.Geometry.Point(-81.357, 30.292), 0, ee.Geometry.Point(-81.651, 30.327)],
    'Charlotte':[ee.Geometry.Point(-80.797, 35.028), ee.Geometry.Point(-80.573, 35.63), ee.Geometry.Point(-80.842, 35.218)],
    'Philadelphia':[ee.Geometry.Point(-75.133, 39.938), 0, ee.Geometry.Point(-75.154, 39.939)],
    'San_Diego':[ee.Geometry.Point(-117.088, 32.647), 0, ee.Geometry.Point(-117.093, 32.578)],
    'San_Juan':[ee.Geometry.Point(-66.157, 18.392), 0, ee.Geometry.Point(-66.066, 18.407)],
    'Montreal':[ee.Geometry.Point(-73.589, 45.523), 0, ee.Geometry.Point(-73.59, 45.52)],
    'Guadalajara':[ee.Geometry.Point(-103.35, 20.663), 0, ee.Geometry.Point(-103.35, 20.663)],
    'Monterrey':[ee.Geometry.Point(-100.17, 25.897), 0, ee.Geometry.Point(-100.318, 25.711)],
    'Cancun':[ee.Geometry.Point(-86.592, 21.402), ee.Geometry.Point(-86.725, 20.709), ee.Geometry.Point(-86.85, 21.161)],
    'Billings':[ee.Geometry.Point(-108.5, 45.78), 0, ee.Geometry.Point(-108.504, 45.781)],
    'Guatemala_City':[ee.Geometry.Point(-90.527, 14.623), 0, ee.Geometry.Point(-90.527, 14.623)],
    'San_Jose':[ee.Geometry.Point(-84.082, 9.937), 0, ee.Geometry.Point(-84.082, 9.937)],
    'Havana':[ee.Geometry.Point(-82.36, 23.128), 0, ee.Geometry.Point(-82.36, 23.128)],
    'Santo_Domingo':[ee.Geometry.Point(-69.822, 18.681), 0, ee.Geometry.Point(-69.932, 18.468)],
    'Tegucigalpa':[ee.Geometry.Point(-87.206, 14.171), 0, ee.Geometry.Point(-87.197, 14.057)],
    'Managua':[ee.Geometry.Point(-86.269, 12.017), ee.Geometry.Point(-86.145, 12.349), ee.Geometry.Point(-86.239, 12.097)],
    'Panama_City':[ee.Geometry.Point(-79.522, 8.98), 0, ee.Geometry.Point(-79.522, 8.98)],
    'Bogota':[ee.Geometry.Point(-74.074, 4.697), 0, ee.Geometry.Point(-74.074, 4.697)],
    'Lima':[ee.Geometry.Point(-76.887, -11.835), ee.Geometry.Point(-76.99, -12.383), ee.Geometry.Point(-77.065, -12.066)],
    'Quito':[ee.Geometry.Point(-78.507, -.232), 0, ee.Geometry.Point(-78.507, -0.232)],
    'Santiago':[ee.Geometry.Point(-70.667, -33.46), 0, ee.Geometry.Point(-70.667, -33.46)],
    'Buenos_Aires':[ee.Geometry.Point(-58.543, -34.621), 0, ee.Geometry.Point(-58.458, -34.666)],
    'Sao_Paulo':[ee.Geometry.Point(-46.644, -23.564), 0, ee.Geometry.Point(-46.644, -23.564)],
    'Manaus':[ee.Geometry.Point(-60.037, -3.089), 0, ee.Geometry.Point(-60.037, -3.089)],
    'Punta_Arenas':[ee.Geometry.Point(-70.184, -53.342), 0, ee.Geometry.Point(-70.909, -53.157)],
    'La_Paz':[ee.Geometry.Point(-67.885, -17.004), ee.Geometry.Point(-67.965, -16.125), ee.Geometry.Point(-68.143, -16.522)],
    'Montevideo':[ee.Geometry.Point(-56.175, -34.902), 0, ee.Geometry.Point(-56.175, -34.902)],
    'Brasilia':[ee.Geometry.Point(-47.890, -15.799), 0, ee.Geometry.Point(-47.89, -15.799)],
    'Caracas':[ee.Geometry.Point(-66.898, 10.475), 0, ee.Geometry.Point(-66.907, 10.474)]
}

In [11]:
# Pull points to make data for a specific city (look above for options)
city = 'Punta_Arenas'
points = city_points[city]

## Landsat Initialization

In [24]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [25]:
landsat = landsat.filterDate('2022-01-01', '2024-01-01')
landsat_US = landsat.filterBounds(points[0])
if points[1] != 0:
    #pt1_filt = ee.Filter.bounds(points[0])
    #pt2_filt = ee.Filter.bounds(points[1])
    #Landsat_pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)
    #landsat_US = landsat.filter(Landsat_pts_filt)
    landsat_US2 = landsat.filterBounds(points[1])
    print('Combine Landsat')
elif points[0] == 0:
    raise Exception('Points not formatted correctly')

In [26]:
landsat_US.first().projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32619',
 'transform': [30, 0, 601785, 0, -30, 1234815]}

In [27]:
# Preprocessing function
def process_multiple_Landsat(image):
    ######################################
    # Combines Landsat images matched by time into one image
    
    match = ee.Image(image.get('matched_img'))
    median_image = ee.ImageCollection([image, match]).median()

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(image.get('system:time_start'))       
        return f.set('system:time_start', Landsat_time)

    # Add timestamp back in
    median_image = get_Landsat_time(median_image)
    
    return median_image

In [28]:
# Join sequential Landsat images
if points[1] != 0:
    best_join = ee.Join.saveBest(matchKey='matched_img', measureKey='time_diff', outer=False)
    # Difference value is equal to 1 minute (for sequential Landsat images)
    timeFiltLandsat = ee.Filter.maxDifference(difference=60e3, leftField='system:time_start', rightField='system:time_start')
    Landsat_near = ee.ImageCollection(best_join.apply(landsat_US, landsat_US2, timeFiltLandsat))
    landsat_US = Landsat_near.map(process_multiple_Landsat)
    print('Landsat join')

In [29]:
print(landsat_US.size().getInfo())

42


## GOES Initialization

In [14]:
# Landsat images over Baltimore regularly taken around 15:47 every 16 days
hour_filt1 = ee.Filter.calendarRange(15, 15, 'hour')
minute_filt_DMV = ee.Filter.calendarRange(45, 49, 'minute') # MUST FIGURE OUT WHEN OTHER LANDSAT IMAGES ARE TAKEN
time_filt1 = ee.Filter.And(hour_filt1, minute_filt_DMV)

hour_filt2 = ee.Filter.calendarRange(3, 3, 'hour')
time_filt2 = ee.Filter.And(hour_filt2, minute_filt_DMV)

time_filt = ee.Filter.Or(time_filt1, time_filt2)

In [30]:
GOES = ee.ImageCollection("NOAA/GOES/16/MCMIPF").filterDate('2022-01-01', '2024-01-01')#.filter(minute_filt_DMV)

In [50]:
gList = GOES.toList(8)

In [51]:
ee.Image(gList.get(0)).date().getInfo()

{'type': 'Date', 'value': 1640995220500}

In [52]:
datetime.fromtimestamp(1640995220500/1000)

datetime.datetime(2021, 12, 31, 19, 0, 20, 500000)

## Sentinel-1 SAR Initialization

In [35]:
sentinel = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2022-01-01', '2024-01-01')

In [36]:
sentinel = sentinel.filterBounds(points[2].buffer(2e5))

In [34]:
print(sentinel.size().getInfo())

65


In [37]:
sentinel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [28444, 23200],
   'crs': 'EPSG:32619',
   'crs_transform': [10, 0, 381431.2080160701, 0, -10, 1435146.4311094424]},
  {'id': 'VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [28444, 23200],
   'crs': 'EPSG:32619',
   'crs_transform': [10, 0, 381431.2080160701, 0, -10, 1435146.4311094424]},
  {'id': 'angle',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [21, 11],
   'crs': 'EPSG:32619',
   'crs_transform': [-12363.661044607405,
    -3941.567629011115,
    664897.1641446773,
    2423.296665806789,
    -20101.69813628611,
    1386332.355583024]}],
 'version': 1741489105472822,
 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20220103T102446_20220103T102513_041295_04E89E_8DC2',
 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '8.0.3',
  'SLC_Processing_facility_org': 'ESA',
  'SLC_Processing_facili

## Filtering images by time

In [38]:
# Join Landsat and Sentinel images
all_join_LS = ee.Join.saveAll(matchesKey='matched_img', measureKey='time_diff', outer=True)
# Difference value is equal to 3 days (Half of Sentinel-1 orbital period) in milliseconds
timeFiltLandsatSentinel = ee.Filter.maxDifference(difference=2592e5, leftField='system:time_start', rightField='system:time_start')
LandsatSentinel_near = ee.ImageCollection(all_join_LS.apply(landsat_US, sentinel, timeFiltLandsatSentinel))

In [39]:
landsat_US.size().getInfo()

42

In [40]:
LandsatSentinel_near.size().getInfo()

42

In [11]:
GOES.size().getInfo()

696

## Helper Functions

In [18]:
# Applies scale and offset factors for Landsat bands
def scale_and_offset_Landsat(image):
    reflect_weight = 0.0000275
    reflect_bias = -0.2
    reflect_image = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(reflect_weight).add(reflect_bias)

    LST_weight = 0.00341802
    LST_bias = 149
    LST_image = image.select('ST_B10').multiply(LST_weight).add(LST_bias)

    return reflect_image.addBands(srcImg=LST_image, names=['ST_B10'])

In [19]:
# Applies scale and offset factors for GOES imagery
# All scale and offset values were taken straight from GEE to save compute
def scale_and_offset_GOES(image):
    #reflectances = image.select(['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06']).multiply(0.0002442)
    weight = 0.039316241
    bias = 173.14999
    brightness_temps = image.select(['CMI_C13', 'CMI_C14', 'CMI_C15', 'CMI_C16']).multiply(weight).add(bias)
    
    #return reflectances.addBands(srcImg=brightness_temps, names=['CMI_C14', 'CMI_C15'])
    return brightness_temps

In [20]:
# Preprocessing function
def process_GOES(image):
    ######################################
    # GOES portion

    # Scaling and offset
    #image2 = ee.Image(feature.get('matched_img'))
    GOES_image = scale_and_offset_GOES(image)

    ######################################
    # Timestamp portion
    
    # Function to get time from GOES image\    
    def get_GOES_time(f):
        GOES_time = ee.Number(image.get('system:time_start'))       
        return f.set('timestamp', GOES_time)

    # Add timestamp back in
    #landsat_image = get_GOES_time(landsat_image)
    GOES_image = get_GOES_time(GOES_image)

    ##########################################
    
    #image = landsat_image.addBands(srcImg=GOES_image, names=['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05',
    #                                                         'CMI_C06', 'CMI_C14', 'CMI_C15']).cast({'QA_PIXEL':'double'})
    return GOES_image

In [21]:
# Preprocessing function
def process_Landsat_Sentinel(feature):
    ######################################
    # Sentinel portion
    if points[3] != 0:
        sentinel_image = ee.Image(ee.ImageCollection(ee.List(feature.get('matched_img'))).median())
    else:
        sentinel_image = ee.Image(ee.List(feature.get('matched_img')).get(0))

    sentinel_image = ee.Algorithms.If(ee.List(feature.get('matched_img')).size().eq(ee.Number(0)),
                    ee.Image(), sentinel_image)
    
    #####################################
    # Landsat portion
    
    # Scaling and offset
    feature = ee.Image(feature)
    #image1 = ee.Image(feature.get('primary'))
    landsat = scale_and_offset_Landsat(feature)
    #landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    landsat_image = landsat.addBands(srcImg=feature, names=['QA_PIXEL'])

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(feature.get('system:time_start'))       
        return f.set('timestamp', Landsat_time)

    # Add timestamp back in
    landsat_image = get_Landsat_time(landsat_image)

    ##########################################
    image = landsat_image.addBands(srcImg=sentinel_image).cast({'QA_PIXEL':'double', 'angle':'double'})
    
    return image

## Produce GOES geotiffs and their respective times

In [22]:
processed = GOES.map(process_GOES)

In [53]:
first = processed.first()

In [54]:
first.getInfo()

{'type': 'Image',
 'bands': [{'id': 'CMI_C13',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -1115.1645950880002,
    'max': 1461.425258847},
   'dimensions': [5424, 5424],
   'crs': 'PROJCS["unnamed", \n  GEOGCS["unknown", \n    DATUM["unknown", \n      SPHEROID["Spheroid", 6378137.0, 298.2572221]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["GEOS"], \n  PARAMETER["central_meridian", -75.0], \n  PARAMETER["satellite_height", 35786023.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["sweep", 0.0], \n  PARAMETER["Option", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
   'crs_transform': [2004.017315487541,
    0,
    -5434894.700982174,
    0,
    2004.017315487541,
    -5434895.218222249]},
  {'id': 'CMI_C14',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    '

In [28]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

In [46]:
times = processed.aggregate_array('timestamp')
features = ee.FeatureCollection(times.map(times_to_features))

In [47]:
task = ee.batch.Export.table.toDrive(
    collection=features,
    description='GOES_times_DMV',
    folder='GOES/GOES_DMV',
    fileFormat='CSV',
)
task.start()

## Produce Landsat/Sentinel geotiffs and their respective times

In [23]:
processed2 = LandsatSentinel_near.map(process_Landsat_Sentinel)

In [82]:
processed2.size().getInfo()

42

In [83]:
processed2.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B6',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B7',
   'data_type': {'ty

In [40]:
times2 = processed2.aggregate_array('timestamp')
features2 = ee.FeatureCollection(times2.map(times_to_features))

In [41]:
task = ee.batch.Export.table.toDrive(
    collection=features2,
    description='Landsat_times_DMV',
    folder='Landsat_SAR_DMV',
    fileFormat='CSV',
)
task.start()

## Export stuff

In [61]:
dt = datetime(year, month, day, hour, minute)
dt

datetime.datetime(2022, 1, 1, 0, 5)

In [62]:
dt.strftime('%Y%m%d%H%M')

'202201010005'

In [ ]:
month = 1
day = 1
year = 2022
hour = -1
minute = 0
for i in range(50):
    if i % 12 == 0:
        hour += 1
        minute = 0
    dt = datetime(year, month, day, hour, minute)
    print(dt.strftime('%Y%m%d%H%M'))
    minute += 5

In [40]:
month = 1
day = 1
year = 2022
hour = -1
minute = 0
for i in range(50):
    if i % 12 == 0:
        hour += 1
        minute = 0
    dt = datetime(year, month, day, hour, minute)
    print(dt.strftime('%Y%m%d%H%M'))
    minute += 5

202201010000
202201010005
202201010010
202201010015
202201010020
202201010025
202201010030
202201010035
202201010040
202201010045
202201010050
202201010055
202201010100
202201010105
202201010110
202201010115
202201010120
202201010125
202201010130
202201010135
202201010140
202201010145
202201010150
202201010155
202201010200
202201010205
202201010210
202201010215
202201010220
202201010225
202201010230
202201010235
202201010240
202201010245
202201010250
202201010255
202201010300
202201010305
202201010310
202201010315
202201010320
202201010325
202201010330
202201010335
202201010340
202201010345
202201010350
202201010355
202201010400
202201010405


In [133]:
"""
Export coordinates for each urban area
KEY: [utm zone, utm x, utm y]
"""
export_coords = {
    'DMV':[18, 292000, 4372200],
    'NYC':[18, 562680, 4550000],
    'Phoenix':[12, 350000, 3749000],
    'Miami':[17, 503991, 2915332],
    'Chicago':[16, 386195, 4680677],
    'Denver':[13, 466349, 4442816],
    'Seattle':[10, 504628, 5311856],
    'San_Francisco':[10, 541523, 4206919],
    'Los_Angeles':[11, 352871, 3803787],
    'Atlanta':[16, 703143, 3787639],
    'Toronto':[17, 566749, 4867878],
    'Mexico_City':[14, 426738, 2202781],
    'Las_Vegas':[11, 618776, 4038562],
    'Salt_Lake_City':[12, 387959, 4539501],
    'Dallas':[14, 639754, 3678675],
    'Houston':[15, 221804, 3334492],
    'New_Orleans':[15, 717043, 3335222],
    'St_Louis':[15, 675752, 4320402],
    'Minneapolis':[15, 433510, 5031071],
    'Jacksonville':[17, 421321, 3375918],
    'Charlotte':[17, 463969, 3944648],
    'Philadelphia':[18, 445906, 4467022],
    'San_Diego':[11, 468102, 3666860],
    'San_Juan':[19, 732591, 2098412],
    'Montreal':[18, 557269, 5097647],
    'Guadalajara':[13, 621576, 2326245],
    'Monterrey':[14, 318956, 2885247],
    'Cancun':[16, 491502, 2369076],
    'Billings':[12, 655301, 5109344],
    'Guatemala_City':[15, 711929, 1659823],
    'San_Jose':[16, 772982, 1148166],
    'Havana':[17, 310663, 2566532],
    'Santo_Domingo':[19, 354586, 2108039],
    'Tegucigalpa':[16, 401921, 1598242],
    'Managua':[16, 524987, 1386278],
    'Panama_City':[17, 610881, 1043658],
    'Bogota':[18, 546553, 562185],
    'Lima':[18, 262348, 8709133],
    'Quito':[17, 730387, 29863],
    'Santiago':[19, 288729, 6352433],
    'Buenos_Aires':[21, 326030, 6204959],
    'Sao_Paulo':[23, 294104, 7446208],
    'Manaus':[20, 762402, 9705737],
    'Punta_Arenas':[19, 353001, 4162220],
    'La_Paz':[19, 548109, 8213756],
    'Montevideo':[21, 545458, 6223320],
    'Brasilia':[22, 799612, 8288848],
    'Caracas':[19, 686519, 1174917]
}

In [134]:
city_export = export_coords[city]
city_export

[16, 491502, 2369076]

In [21]:
# Export GOES images
num = processed.size().getInfo()
#num = 1
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_image_{i}', fileFormat='GeoTIFF', folder='GOES_DMV_2km',
                dimensions='45x45', crs=f'EPSG:326{city_export[0]}', crsTransform=[2000, 0, city_export[1], 0, -2000, city_export[2]])
    task.start()

In [41]:
# Export Landsat/Sentinel images
#num = processed2.size().getInfo()
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{i}', fileFormat='GeoTIFF', folder='Landsat_SAR_DMV_new',
                dimensions='3000x3000', crs=f'EPSG:326{city_export[0]}', crsTransform=[30, 0, city_export[1], 0, -30, city_export[2]])
    task.start()

In [135]:
# Export GOES images for export to GEE asset for visualizing
crs_prefix = '326' # Northern hemisphere
#crs_prefix = '327' # Southern hemisphere
num = 1
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'GOES_image_{i}', assetId=f'projects/ee-jonstar/assets/{city}_GOES_test',
                dimensions='45x45', crs=f'EPSG:{crs_prefix}{city_export[0]}', crsTransform=[2000, 0, city_export[1], 0, -2000, city_export[2]])
    task.start()

In [27]:
# Export Landsat images for export to GEE asset for visualizing
#crs_prefix = '326' # Northern hemisphere
crs_prefix = '327' # Southern hemisphere
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toAsset(
        img, description=f'Landsat_Sentinel_image_{i}', assetId=f'projects/ee-jonstar/assets/{city}_LS_test',
                dimensions='3000x3000', crs=f'EPSG:{crs_prefix}{city_export[0]}', crsTransform=[30, 0, city_export[1], 0, -30, city_export[2]])
    task.start()